# 1. Get the packages

In [319]:
import cohere
import pandas as pd
import requests
import datetime
from tqdm import tqdm

# 2. Get the Data

In [320]:
from cgi import test


train_df = pd.read_json('../data/entity/relations_training.txt')
test_df = pd.read_json('../data/entity/relations_test.txt') 


In [321]:
from cProfile import label


def pre_process_tokens(tokens):
    enti={}
    for t in tokens:
        
        if t['entityLabel'] in enti.keys():
            enti[t['entityLabel']] = enti[t['entityLabel']] +','+t['text']
        else:
            enti[t['entityLabel']] = t['text']
    label = ""
    for k, v in enti.items():
        label += k+':'+v+"\n"
    return label

In [322]:
from random import triangular


train_df['label'] = train_df['tokens'].apply(pre_process_tokens)
test_df['label'] = test_df['tokens'].apply(pre_process_tokens)


In [323]:
test_df

,document,tokens,relations,label
0,"\nCurrently holding a faculty, industry, or go...","[{'text': 'Ph.D.', 'start': 75, 'end': 80, 'to...","[{'child': 18, 'head': 14, 'relationLabel': 'D...","DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:machine learning,..."
1,\n2+ years experience in the online advertisin...,"[{'text': '2+ years', 'start': 1, 'end': 9, 't...","[{'child': 7, 'head': 1, 'relationLabel': 'EXP...",EXPERIENCE:2+ years\nSKILLS:online advertising...
2,\nBA/BS\n5+ years of program or project manage...,"[{'text': '5+ years', 'start': 7, 'end': 15, '...","[{'child': 11, 'head': 5, 'relationLabel': 'EX...","EXPERIENCE:5+ years,2+ years\nSKILLS:project m..."
3,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
4,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
5,\nMS in a quantitative field such as Operation...,"[{'text': 'MS', 'start': 1, 'end': 3, 'token_s...","[{'child': 8, 'head': 1, 'relationLabel': 'DEG...","DIPLOMA:MS,PhD\nDIPLOMA_MAJOR:Operations Resea..."
6,"\nPh.D. with 5+ years of experience, MS with 7...",[],[],
7,\nGraduating with a Ph.D. in Computer Science ...,"[{'text': 'Ph.D.', 'start': 19, 'end': 24, 'to...","[{'child': 6, 'head': 4, 'relationLabel': 'DEG...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
8,"\n12+ years of research experience, in-house o...","[{'text': '12+ years', 'start': 1, 'end': 10, ...","[{'child': 5, 'head': 1, 'relationLabel': 'EXP...","EXPERIENCE:12+ years\nSKILLS:research,Ph.D\nDI..."
9,"\nPhD with 3+ years, or MS with 5+ years of ex...","[{'text': '5+ years', 'start': 31, 'end': 39, ...","[{'child': 18, 'head': 10, 'relationLabel': 'E...","EXPERIENCE:5+ years,3+ years,3+ years,5+ years..."


# 3. preprocess the data

In [324]:

train_doc = []
for i in range(train_df.shape[0]):
    ent = train_df.label.iloc[i]
    docu = train_df.document.iloc[i].replace("\n"," ")
    if len(ent)!=0:
        train_doc.append(docu+"\n\nExtracted Text:" +
                        '\n'+ent+"----\n")
    
with open('../data/output/entity.txt','w') as f:
    for item in train_doc:
        # write each item on a new line
        f.write("%s\n" % item.strip())
    print('Done')
    

Done


In [325]:

test_doc = test_df.document.apply(
    lambda x: x.replace("\n", " ")+'\n\nExtracted Text:').to_list()


In [326]:
import cohere
api_key = pd.read_json("../config/apikey.json")['cohere_key'][0]
co = cohere.Client(api_key)


# 4. Running the model

In [327]:
#@title Create the prompt (Run this cell to execute required code) {display-mode: "form"}

class cohereExtractor():
    def __init__(self, examples):
        self.examples = examples

    def make_prompt(self, example):
        prompt = self.examples + [example]
       
        return ("".join([str(exam) for exam in prompt]))

    def extract(self, example):
        extraction = co.generate(
            model='large',
            prompt=self.make_prompt(example),
            max_tokens=100,
            temperature=0.5,
            stop_sequences=["----"])
        
        return(extraction.generations[0].text[:-1])


cohereMovieExtractor = cohereExtractor(train_doc[:2])



In [328]:
print(cohereMovieExtractor.make_prompt(test_doc[-1]))


MS/Ph.D. in CS/EE or related areas is required. Strong ability and effectiveness working in a significant technical problem domain, in the term of plan, design, execution, continuous release and service operation. Strong software engineering fundamentals, including coding, problem solving and data analysis skills. Background in machine learning/deep learning (strongly preferred). Passionate and self-motivated. Ability to effectively work in collaborative multiple project team environment and ship production features in a fast-paced environment. Good communication skills, both verbal and written. Customer/end result/Metrics driven in design and development. Strong ability in self-learning, entering new domain, managing through uncertainty in an innovative team environment

Extracted Text:
DIPLOMA:MS/Ph.D.
DIPLOMA_MAJOR:CS/EE
----
1+ years development experience on Java stack AppConnect / API's experience is added advantage. Compute, Network and Storage Monitoring Tools (Ex: Netcool) App

In [329]:
results = []
for text in tqdm(test_doc[-10:]):
    try:
        extracted_text = cohereMovieExtractor.extract(text)
        print(extracted_text)
        results.append(extracted_text)
    except Exception as e:
        print('ERROR: ', e)


 10%|█         | 1/10 [00:03<00:35,  3.98s/it]


EXPERIENCE:2+ years
SKILLS:online advertising
---


In [ ]:
pd.DataFrame(data={'text': test_doc[-10:], 'extracted_text': results})


,text,extracted_text
0,2+ years experience in the online advertising...,\nEXPERIENCE:2+ years\nSKILLS:online advertisi...
1,BA/BS 5+ years of program or project manageme...,\nEXPERIENCE:5+ years\nSKILLS:program\n---
2,Currently enrolled in a full-time degree prog...,\nRESEARCH INTERNSHIP: Currently enrolled in a...
3,Currently enrolled in a full-time degree prog...,\nSKILLS:research\n---
4,MS in a quantitative field such as Operations...,\nDIPLOMA:MS/Ph.D.\nDIPLOMA_MAJOR:QUANTITATIVE...
5,"Ph.D. with 5+ years of experience, MS with 7+...","\nPh.D. with 5+ years of experience, MS with 7..."
6,Graduating with a Ph.D. in Computer Science o...,"\nEXPERIENCE:0-1 years\nSKILLS:C++,Python,C#,J..."
7,"12+ years of research experience, in-house or...",\nDIPLOMA:MS/Ph.D.\nDIPLOMA_MAJOR:CS/EE\n---
8,"PhD with 3+ years, or MS with 5+ years of exp...","\nEXPERIENCE:3+ years\nSKILLS:research, manage..."
9,"Currently holding a faculty, industry, or gov...",\nEXPERIENCE:1+ years\nSKILLS:research\n---


In [ ]:
def clean_text(x):
    x = x.lstrip()
    x = ''.join(x.split())
    x = x.lower()
    return x


In [ ]:
from concurrent.futures import ThreadPoolExecutor

extracted = []
# Run the model to extract the entities
with ThreadPoolExecutor(max_workers=8) as executor:
    for i in executor.map(cohereMovieExtractor.extract, test_df['document'].apply(
            lambda x: "Job: "+x.replace("\n", " ")+'\n\nExtracted Text:').to_list()):
        extracted.append(str(i).strip())



In [ ]:
# Save results
test_df['extracted_text'] = extracted


In [ ]:
test_df['extracted_text'] = test_df['extracted_text'].apply(clean_text)
# test_df['label'] = test_df['label'].apply(clean_text)


In [ ]:
# Compare the label to the extracted text
test_df['correct'] = ( test_df['label'] == test_df['extracted_text']).astype(int)

# Print the accuracy
print(f'Classification accuracy {test_df["correct"].mean() *100}%')


Classification accuracy 0.0%


In [ ]:
test_df[test_df['correct']==0]

,document,tokens,relations,label,extracted_text,correct
0,"\nCurrently holding a faculty, industry, or go...","[{'text': 'Ph.D.', 'start': 75, 'end': 80, 'to...","[{'child': 18, 'head': 14, 'relationLabel': 'D...","diploma:ph.d.diploma_major:machinelearning,ai,...","research:currentlyholdingafaculty,industry,org...",0
1,\n2+ years experience in the online advertisin...,"[{'text': '2+ years', 'start': 1, 'end': 9, 't...","[{'child': 7, 'head': 1, 'relationLabel': 'EXP...","experience:2+yearsskills:onlineadvertising,res...",experience:2+yearsskills:analytics---,0
2,\nBA/BS\n5+ years of program or project manage...,"[{'text': '5+ years', 'start': 7, 'end': 15, '...","[{'child': 11, 'head': 5, 'relationLabel': 'EX...","experience:5+years,2+yearsskills:projectmanage...",experience:5+yearsskills:program---,0
3,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",diploma:ph.d.diploma_major:computerscience,experience:currentlyenrolledinafull-timedegree...,0
4,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",diploma:ph.d.diploma_major:computerscience,diploma:ms/ph.d.diploma_major:cs/ee---,0
5,\nMS in a quantitative field such as Operation...,"[{'text': 'MS', 'start': 1, 'end': 3, 'token_s...","[{'child': 8, 'head': 1, 'relationLabel': 'DEG...","diploma:ms,phddiploma_major:operationsresearch...",experience:7+yearsskills:analysis---,0
6,"\nPh.D. with 5+ years of experience, MS with 7...",[],[],,job:ph.d.with5+yearsofexperiencejob_mode:,0
7,\nGraduating with a Ph.D. in Computer Science ...,"[{'text': 'Ph.D.', 'start': 19, 'end': 24, 'to...","[{'child': 6, 'head': 4, 'relationLabel': 'DEG...",diploma:ph.d.diploma_major:computerscience,experience:1+yearsskills:development---,0
8,"\n12+ years of research experience, in-house o...","[{'text': '12+ years', 'start': 1, 'end': 10, ...","[{'child': 5, 'head': 1, 'relationLabel': 'EXP...","experience:12+yearsskills:research,ph.ddiploma...",experience:12+yearsskills:research---,0
9,"\nPhD with 3+ years, or MS with 5+ years of ex...","[{'text': '5+ years', 'start': 31, 'end': 39, ...","[{'child': 18, 'head': 10, 'relationLabel': 'E...","experience:5+years,3+years,3+years,5+yearsskil...",experience:3+yearsskills:research---,0


In [ ]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

# print(classification_report(test_df['label'], test_df['extracted_text']))
